# Search Results Pre-processing
This notebook aims to clean up the data from teh search term results across all databases used for the systematic review on audiometric data and UML.

You can find the databases, search terms and number of returns under `~/README.md`

## Necessary packages
Outside of the usual python packages (numpy, pandas, etc.), you will need `xlrd`, `crossrefapi` and `bibtexparser`. Install using:

`pip install xlrd`

`pip install crossrefapi`

`pip install bibtexparser`

## Import modules and define functions

In [3]:
# import modules
import os
from tqdm import tqdm

import numpy as np
import pandas as pd

import re
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import bibtexparser
from crossref.restful import Works

from IPython.display import display, Markdown, clear_output

In [4]:
def load_bib(path_to_bib):
    with open(path_to_bib) as bibtex_file:
        bib_database = bibtexparser.load(bibtex_file)

    # Convert the bibtex entries to a list of dictionaries
    bib_list = bib_database.entries

    # Convert the list of dictionaries to a pandas DataFrame
    bib_df = pd.DataFrame(bib_list)
    
    return bib_df

def load_xls(path_to_xls, n_skip_rows):
    xls_df = pd.read_excel(path_to_xls, engine='xlrd', skiprows=n_skip_rows)
    return xls_df

def standardised_df(df, title, abstract, database_handle, database_id, doi):
    # check if database_id is None
    if database_id == None:
        database_id_series = np.full(df.shape[0], None)
    else:
        database_id_series = df[database_id]
    
    # check if abstract is None
    if abstract == None:
        abstract_series = np.full(df.shape[0], None)
    else:
        abstract_series = df[abstract]

    # check if doi is None
    if doi == None:
        doi_series = np.full(df.shape[0], None)
    else:
        doi_series = df[doi]
    standardised_df = pd.DataFrame({
        'title': df[title],
        'abstract': abstract_series,
        'database': np.full(df.shape[0], database_handle),
        'database_id': database_id_series,
        'doi': doi_series
    })

    return standardised_df

# Function to split abstracts
# PubMed
def split_pubmed_actracts(file_path, separator):
    """
    Splits a text file into strings based on a specific separator.

    Args:
        file_path (str): Path to the text file.
        separator (str): Separator to split the file content.

    Returns:
        dict: Dictionary containing split strings with indices as keys.
    """
    result = {}
    with open(file_path, 'r') as file:
        content = file.read()
        split_strings = re.split(separator, content)
        for idx, s in enumerate(split_strings):
            if idx == 0:
                clean_s = s
            else:
                # Remove irrelevant header
                clean_s = ''.join(s.split('\n')[1:])
                try:
                    # Find the index of the first digit (start of chunk)
                    start_index = next(i for i, char in enumerate(clean_s) if char.isdigit())
                    clean_s = clean_s[start_index:].strip()
                except StopIteration:
                    # If no digit found, keep the entire chunk
                    pass
            
            result[idx] = clean_s.strip()
    return result

# Function to find rough duplicates based on cosine similarity of TF-IDF vectors
def find_rough_duplicates(data, column='title', threshold=0.8):
    # Vectorize the titles using TF-IDF
    vectorizer = TfidfVectorizer().fit_transform(data[column])
    
    # Compute cosine similarity matrix
    cos_sim = cosine_similarity(vectorizer)
    
    # Find indices where similarity is above the threshold, ignoring self-similarity (diagonal)
    duplicates = np.triu(np.where((cos_sim > threshold) & (cos_sim < 1), 1, 0), 1)
    
    # Extract indices of duplicates (only need one index per pair, so we choose the first)
    duplicate_indices = np.unique(np.nonzero(duplicates)[0])
    
    # Drop the identified duplicates
    data_deduped = data.drop(index=duplicate_indices)
    
    return data_deduped, len(duplicate_indices)

def identify_rough_duplicate_pairs(data, column='title', threshold=0.8):
    # Vectorize the titles using TF-IDF
    vectorizer = TfidfVectorizer().fit_transform(data[column])
    
    # Compute cosine similarity matrix
    cos_sim = cosine_similarity(vectorizer)
    
    # Find indices where similarity is above the threshold, ignoring self-similarity (diagonal)
    duplicates_matrix = np.triu(np.where((cos_sim > threshold) & (cos_sim < 1), 1, 0), 1)
    
    # Extract pairs of indices that are considered duplicates
    duplicate_indices_pairs = np.column_stack(np.where(duplicates_matrix > 0))
    
    # Extract the titles of these duplicate pairs along with their similarity score
    duplicate_pairs = [(data.iloc[pair[0]][column], data.iloc[pair[1]][column], cos_sim[pair[0], pair[1]]) for pair in duplicate_indices_pairs]
    
    return duplicate_pairs

def jats_to_markdown(text):
    if pd.isnull(text):  # Skip conversion if text is NaN
        return text
    # Convert section tags by removing them, assuming sections are already implied or handled by other means
    text = re.sub(r'<\/?jats:sec>', '', text)  # Remove opening and closing section tags without replacement
    # Handle titles within sections (assuming bold in Markdown for titles)
    text = re.sub(r'<jats:title>(.*?)</jats:title>', r'\n**\1**\n', text)
    # Convert paragraphs
    text = re.sub(r'<jats:p>(.*?)</jats:p>', r'\n\1\n', text)
    # Convert bold and italic text
    text = re.sub(r'<jats:bold>(.*?)</jats:bold>', r'**\1**', text)
    text = re.sub(r'<jats:italic>(.*?)</jats:italic>', r'*\1*', text)
    # Convert links - assuming 'href' is the attribute for URL
    text = re.sub(r'<jats:ext-link href="(.*?)">(.*?)</jats:ext-link>', r'[\2](\1)', text)
    # Handle lists
    text = re.sub(r'<jats:list list-type="bullet">(.*?)</jats:list>', r'\n\1\n', text)
    text = re.sub(r'<jats:list-item>(.*?)</jats:list-item>', r'- \1', text)
    # General cleanup for any remaining JATS tags (opening and closing)
    text = re.sub(r'<\/?jats\:.*?>', '', text)
    return text.strip()

def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\W', ' ', text)  # Remove all non-word characters
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    # Apply stemming
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    text = ' '.join(stemmed_words)
    return text

## Loading and joining data

In [3]:
# load in data and standardise
# PUBMED
pubmed_df = pd.read_csv('../data/search_returns/medline.csv')
pubmed_standard_df = standardised_df(pubmed_df, 'Title', None, 'pubmed', 'PMID', 'DOI')

# Scopus
scopus_df = pd.read_csv('../data/search_returns/scopus.csv')
scopus_standard_df = standardised_df(scopus_df, 'Title', 'Abstract', 'scopus', 'EID', 'DOI')

# PsycINFO
psychinfo_df = pd.read_csv('../data/search_returns/psychinfo.csv')
psychinfo_standard_df = standardised_df(psychinfo_df, 'TI', 'AB', 'psychinfo', None, 'URL')

# EMBASE
embase_df = pd.read_csv('../data/search_returns/embase.csv')
embase_standard_df = standardised_df(embase_df, 'TI', 'AB', 'embase', 'UI', 'FTURL')

# CINAHL
cinahl_df = pd.read_csv('../data/search_returns/cinahl.csv')
cinahl_standard_df = standardised_df(cinahl_df, 'artinfo/tig/atl', 'artinfo/ab/0', 'cinahl', 'jinfo/issn', 'artinfo/ui/0/__text')

# IEEE
ieee_df = pd.read_csv('../data/search_returns/ieee.csv')
ieee_standard_df = standardised_df(ieee_df, 'Document Title', 'Abstract', 'ieee', 'ISSN', 'DOI')

# BASE
base_df = pd.read_csv('../data/search_returns/base.csv')
base_standard_df = standardised_df(base_df, 'title', None, 'base', 'ID', None)

Add in abstracts

In [4]:
# pubmed
file_path = "../data/search_returns/pubmed_abstracts.txt"
separator = "PMID"
split_strings_dict = split_pubmed_actracts(file_path, separator)

last_id = max(split_strings_dict.keys())
del split_strings_dict[last_id]

abstract_series = pd.Series(split_strings_dict)
abstract_series.reset_index(inplace=True, drop=True)

pubmed_standard_df.abstract = abstract_series

In [ ]:
pubmed_standard_df

In [28]:
# join standardised dataframes
standard_db_df = pd.concat([
    pubmed_standard_df, psychinfo_standard_df, scopus_standard_df, cinahl_standard_df, ieee_standard_df, base_standard_df
], axis=0)
standard_db_df.reset_index(inplace=True, drop=True)

In [29]:
# remove special characters from title
titles_cleaned = standard_db_df.title.str.replace('[^a-zA-Z0-9\s]', '', regex=True)
standard_db_df.title = titles_cleaned

standard_db_df['title'] = standard_db_df['title'].str.lower()

In [ ]:
standard_db_df

In [31]:
# number of missing abstracts
sum(standard_db_df.abstract.isnull())

109

In [178]:
# export this data for future use
standard_db_df.to_csv('../data/search_returns/all_db.csv', index=False)

## De-duplicate data

In [32]:
# deduplicate on exact matches
standard_db_deduped_exact = standard_db_df.drop_duplicates(subset=['title'], keep='first')
standard_db_deduped_exact.reset_index(inplace=True, drop=True)


# drop row if title is NaN
standard_db_deduped_exact = standard_db_deduped_exact.dropna(subset=['title'])

# Display the shape of the original and deduplicated dataframes to see how many rows were removed
original_shape = standard_db_df.shape
deduped_exact_shape = standard_db_deduped_exact.shape


original_shape, deduped_exact_shape


((1972, 5), (1402, 5))

In [ ]:
standard_db_deduped_exact

In [34]:
# deduplicate on fuzzy matches
# Apply function to find and drop rough duplicates
data_deduped_rough, num_rough_duplicates = find_rough_duplicates(standard_db_deduped_exact, 'title', 0.814)

# Display the number of rough duplicates removed and the new shape of the dataframe
num_rough_duplicates, data_deduped_rough.shape


(59, (1343, 5))

In [35]:
# Identify rough duplicate pairs in the DataFrame before removing them
rough_duplicate_pairs = identify_rough_duplicate_pairs(standard_db_deduped_exact, 'title', 0.814)

# Convert the list of rough duplicate pairs into a pandas DataFrame for better readability and presentation
rough_duplicate_pairs_df = pd.DataFrame(rough_duplicate_pairs, columns=['Title 1', 'Title 2', 'Similarity'])

In [ ]:
rough_duplicate_pairs_df

In [ ]:
data_deduped_rough

## Add in missing abstract info where possible

In [37]:
works = Works()

doi_column = 'doi'
abstract_column = 'abstract'
rows = list(data_deduped_rough.iterrows())

# Use tqdm to wrap around the iterrows loop for a progress bar
for index, row in tqdm(rows, desc="Updating abstracts"):
    # Check if there's no abstract but a DOI is available
    if pd.isnull(row[abstract_column]) and pd.notnull(row[doi_column]):
        try:
            # Attempt to retrieve the work from Crossref using the DOI
            work = works.doi(row[doi_column])
            # If an abstract is found, update the DataFrame
            if work and 'abstract' in work:
                data_deduped_rough.at[index, abstract_column] = work['abstract']
        except Exception as e:
            print(f"Error retrieving abstract for DOI {row[doi_column]}: {str(e)}")


Updating abstracts: 100%|██████████| 1343/1343 [00:03<00:00, 391.55it/s] 


In [38]:
# tidy up new abstracts to markdown
data_deduped_rough['abstract'] = data_deduped_rough['abstract'].apply(jats_to_markdown)


In [ ]:
data_deduped_rough

In [40]:
# number of missing abstracts
sum(data_deduped_rough.abstract.isnull())

45

## Rank the data by similarity to exemplar papers

In [42]:
# import data
exemplar_df = pd.read_csv('../data/search_returns/exemplar_papers.csv')

stemmer = PorterStemmer()

# Combine titles and abstracts, with handling for missing abstracts
combined_texts = data_deduped_rough['title'] + " " + data_deduped_rough['abstract'].fillna('no abstract')
exemplar_combined_texts = exemplar_df['title'] + " " + exemplar_df['abstract'].fillna('no abstract')

# Apply preprocessing
combined_texts = combined_texts.apply(preprocess_text)
exemplar_combined_texts = exemplar_combined_texts.apply(preprocess_text)

vectorizer = TfidfVectorizer()
all_texts = pd.concat([combined_texts, exemplar_combined_texts])
vectorized_texts = vectorizer.fit_transform(all_texts)

num_dataset_docs = len(data_deduped_rough)
num_exemplar_docs = len(exemplar_df)
cosine_similarities = cosine_similarity(vectorized_texts[:num_dataset_docs], vectorized_texts[num_dataset_docs:])

average_similarities = np.mean(cosine_similarities, axis=1)
ranked_indices = np.argsort(-average_similarities)  # Negate for descending order
ranked_similarities = average_similarities[ranked_indices]


In [43]:
data_deduped_rough['similarity_score'] = average_similarities

# Step 2: Sort the DataFrame by similarity scores in descending order
data_deduped_rough_sorted = data_deduped_rough.sort_values(by='similarity_score', ascending=False)

# Optional: Add a ranking column based on the sorted order
data_deduped_rough_sorted['rank'] = range(1, len(data_deduped_rough_sorted) + 1)

In [ ]:
data_deduped_rough_sorted.iloc[:10, :]

In [ ]:
data_deduped_rough_sorted.iloc[-10:, :]

In [46]:
data_deduped_rough_sorted.reset_index(inplace=True, drop=True)
data_deduped_rough_sorted.to_csv('../data/tiab/all_results_deduplicated_ordered.csv', index=False)